In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


ModuleNotFoundError: No module named 'geopandas'

In [ ]:
housing_df = pd.read_csv("melb_data.csv")

In [ ]:
#housing_df['Type']
housing_df.loc[(housing_df['Type']=='h') | (housing_df['Type']=='t'),:]

In [ ]:
housing_df.isna().sum()

In [ ]:
housing_df.describe()

In [ ]:
housing_df['Date'].unique()

In [ ]:
housing_cleaned = housing_df.copy()
housing_cleaned.loc[:,'Date'] = pd.to_datetime(housing_cleaned.loc[:,'Date'], format="%d/%m/%Y")
print(housing_cleaned['Date'].min())
print(housing_cleaned['Date'].max())


In [ ]:
housing_cleaned

In [ ]:
housing_dropped = housing_cleaned.copy()
housing_dropped = housing_dropped.drop(columns=['CouncilArea'])
housing_dropped = housing_dropped.dropna()
housing_dropped.dtypes

In [ ]:
numeric_cols = housing_dropped.select_dtypes(include=['float64']).columns
print(numeric_cols)
corr_matrix = housing_dropped.loc[:,numeric_cols].corr()

In [ ]:
#Only use houses and townhouses (no apartments)
housing_dropped = housing_dropped.loc[(housing_dropped['Type']=='h') | (housing_dropped['Type']=='t'),:]

In [ ]:
sns.histplot(data=housing_dropped['Price'])

In [ ]:
housing_dropped.describe()

In [ ]:
housing_dropped['Price'].min()

In [ ]:
housing_dropped['Price'].max()

In [ ]:
len(pd.unique(housing_dropped['Suburb']))

In [ ]:
housing_dropped.Suburb.value_counts()

In [ ]:
# Combine the shapefiles